In [3]:
import os

In [4]:
from modules.pipeline import *

modules.report successfull loaded: 100%|██████████| 6/6 [00:07<00:00,  1.26s/it]


In [5]:
from unidecode import unidecode

In [6]:
import re

In [7]:
def get_filenames(root_dir, func, verbose=False):
    data_filenames = []
    # Walk through the directories and files
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # dirpath: current directory path
        # dirnames: list of directories in the current directory
        # filenames: list of files in the current directory

        # Print the current directory
        print('Directory:', dirpath)  if verbose else None
        # Print all the subdirectories
        if verbose:
            for dirname in dirnames:
                print('Subdirectory:', os.path.join(dirpath, dirname))

        # Print all the files
        for filename in filenames:
            if func(filename) and not ('x_' in filename or 'y_' in filename or 'metric' in filename):
                print('File:', os.path.join(dirpath, filename)) if verbose else None
                data_filenames.append(os.path.join(dirpath, filename))

        # Print an empty line to separate directories
        print()  if verbose else None
    return data_filenames

In [8]:
MLN_F = lambda x: (('classic_mln_' in x) and not('classic_mln_-' in x)) # find metric of model where mln were added
MLN__F = lambda x: (('classic_mln_-' in x)) # where mln attribut were removed first
MLN_C_F = lambda x: (('classic_-' in x)) # where mln attribut were removed first
MLN_C= lambda x: (('classic_' in x)) # where mln attribut were removed first

INTER_F = lambda x: (not('_max_' in x) and ('inter' in x))
INTRA_F = lambda x: (not('_max_' in x) and ('intra' in x))
COMBINE_F = lambda x: (not('_max_' in x) and ('combine' in x))
ULTRA_F = lambda x: (not('_max_' in x) and ('ultra' in x))
INTER_MAX_F = lambda x: (('_max_' in x) and ('inter' in x))
INTRA_MAX_F = lambda x: (('_max_' in x) and ('intra' in x))
COMBINE_MAX_F = lambda x: (('_max_' in x) and ('combine' in x))
ULTRA_MAX_F = lambda x: (('_max_' in x) and ('ultra' in x))
DEGREE_F = lambda x: (('degree' in x))

In [9]:
def metrics_analyzer(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    outputs = {}
    
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer
        for k in list(set([1, 2, len(cols)])):
            #for logic in ['global', 'personalized']:
            #outputs[logic] = {}
            ### get all combination of col
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                if sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode('cb_person_default_on_file±loan_intent'))
                            for partern in case_k.split("±")
                            ]
                        ) == k: # check mission context
                    continue
                print(case_k)
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                models = {}
                models_1 = {}
                for model in models_list:
                    data = {
                        'global':{},
                        'personalized':{}
                    }
                    for key in files['global'].keys():
                        data['global'][key] = [files['global'][key].loc[model,metric] for metric in metrics]
                        data['personalized'][key] = [files['personalized'][key].loc[model,metric] for metric in metrics]

                    data['global']['Metrics'] = metrics
                    data['personalized']['Metrics'] = metrics
                    models[model] = pd.DataFrame(data=data['global'])
                    models_1[model] = pd.DataFrame(data=data['personalized'])
                ### generate figures
                nrow = 2
                ncol = 4
                
                fig, axs = plt.subplots(nrows=nrow, ncols=ncol, figsize=(20, 15))
                count=0
                for r in range(ncol): 
                    # Barplot i
                    models[models_list[count]].plot(kind='bar', x='Metrics', y=files['global'].keys(), ax=axs[0,r])
                    axs[0,r].set_title(f'{models_name[models_list[count]]}')
                    axs[0,r].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                    
                    models_1[models_list[count]].plot(kind='bar', x='Metrics', y=files['personalized'].keys(), ax=axs[1,r])
                    axs[1,r].set_title(f'{models_name[models_list[count]]}')
                    axs[1,r].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                    
                    axs[0,r].legend().set_visible(False)
                    axs[1,r].legend().set_visible(False)
                    if r == ncol-1:
                        axs[1,r].legend().set_visible(True)
                    count+=1
                axs[0,0].set_ylabel('global')
                axs[1,0].set_ylabel('personalized')
                if True:
                    create_domain(f'{cwd}/analyser/{data_folder}/plots/mlna_{k}/mixed')

                    timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
                    filename1 = f'{cwd}/analyser/{data_folder}/plots/mlna_{k}/mixed/_metrics_comparaison_for_{case_k}'+'_'+timestr+'.png'
                    # Adjust the layout to cover all content
                    plt.tight_layout()

                    plt.savefig(filename1,dpi=150) #.png,.pdf will also support here
                    plt.close() # close the plot windows
                ### generate reports
    return outputs



In [10]:
# @profile
def print_summary(table_list, modelDict):
    baseline = table_list[0][1].index.values.tolist()
    head = '<tr><td></td><td></td><td>Accuracy</td>'+'<td>Precision</td><td>Recall</td><td>F1-score</td></tr>'
    body = ''
    for model in baseline:
        modelLines = f'<tr><td rowspan="{len(table_list)}" >{modelDict[model]}</td>'
        for i, (step,data) in enumerate(table_list):
            modelLines = (
                modelLines + 
                f'<td>{step}</td><td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"accuracy"],4) == max(round(dat.loc[model,"accuracy"],4),round(data.loc[model,"accuracy"],4)) for step, dat in table_list])/len(table_list))} >{round(data.loc[model,"accuracy"],4)}</td>'+
                f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"precision"],4) == max(round(dat.loc[model,"precision"],4),round(data.loc[model,"precision"],4)) for step, dat in table_list])/len(table_list))}>{round(data.loc[model,"precision"],4)}</td>'+
                f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"recall"],4) == max(round(dat.loc[model,"recall"],4),round(data.loc[model,"recall"],4)) for step, dat in table_list])/len(table_list))}>{round(data.loc[model,"recall"],4)}</td>'+
                f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"f1-score"],4) == max(round(dat.loc[model,"f1-score"],4),round(data.loc[model,"f1-score"],4)) for step, dat in table_list])/len(table_list))}>{round(data.loc[model,"f1-score"],4)}</td>'+
                '</tr> <tr >') if i < len(table_list) - 1 else (
                modelLines + 
                f'<td>{step}</td><td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"accuracy"],4) == max(round(dat.loc[model,"accuracy"],4),round(data.loc[model,"accuracy"],4)) for step, dat in table_list])/len(table_list))} >{round(data.loc[model,"accuracy"],4)}</td>'+
                f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"precision"],4) == max(round(dat.loc[model,"precision"],4),round(data.loc[model,"precision"],4)) for step, dat in table_list])/len(table_list))}>{round(data.loc[model,"precision"],4)}</td>'+
                f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"recall"],4) == max(round(dat.loc[model,"recall"],4),round(data.loc[model,"recall"],4)) for step, dat in table_list])/len(table_list))}>{round(data.loc[model,"recall"],4)}</td>'+
                f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(sum([ round(data.loc[model,"f1-score"],4) == max(round(dat.loc[model,"f1-score"],4),round(data.loc[model,"f1-score"],4)) for step, dat in table_list])/len(table_list))}>{round(data.loc[model,"f1-score"],4)}</td>'+
                '</tr>' )
        body = body + modelLines
    style = '<style>table, th, td {border: 1px solid black;border-collapse: collapse;}</style>'
    table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{head}{body}</table>'
    htm = f'<html><head>{style}<title> Summary </title></head><body style="background-color: white;">{table_html}</body></html>'
    return (HTML(htm),htm)

# @profile
def create_file(content, cwd, filename, extension=".html", prefix=None):
    """
    """

    create_domain(cwd+'/reports/')
    timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
    filename1 = cwd+'/reports'+'/'+filename+timestr+extension
    _file= open(filename1,"w")
    _file.write(content)
    _file.close()

    return filename1

In [11]:
def metrics_analyzer(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer
        head_lambda = lambda x: f'<tr><td rowspan="2" colspan="3">{x}</td><td colspan="4">Classic</td><td colspan="4">Classic - Att</td><td colspan="4">Classic + mln</td><td colspan="4">Classic + mln - Att</td></tr><tr class="metrics">{"<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>" * 4}</tr>'
        head = {
            'xgb': head_lambda('xgb'),
            'dtc': head_lambda('dtc'),
            'lrc': head_lambda('lrc'),
            'rfc': head_lambda('rfc')
        }
        body = {
            'xgb': '',
            'dtc': '',
            'lrc': '',
            'rfc': ''
        }
        style = '<style>tr:nth-child(even) {background-color: #dddddd;} table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            f'{x}': models_name[x],
            'MLN': 'MultiLayer Network',
            'MLN k Layer(s)': 'MLN with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'Desc': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + mln': f'Learning from classic dataset of {data_folder} where mln had been added',
            'Classic + mln - Att': f'Learning from classic dataset of {data_folder} where mln had been added and Att removed'
            }.items()])

        for k in list(set([1, 2, len(cols)])):
            #for logic in ['global', 'personalized']:
            #outputs[logic] = {}
            ### get all combination of col
            body_l = {
                'xgb': '',
                'dtc': '',
                'lrc': '',
                'rfc': ''
            }
            LayerLines = f'<tr style="border-top: 2px solid black;"><td rowspan="{len(get_combinations(range(len(cols)),k)) * 2}" >MLN {k} layer(s)</td>'
            
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                #if sum(
                #        [
                #            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode('cb_person_default_on_file±loan_intent'))
                #            for partern in case_k.split("±")
                #            ]
                #        ) == k: # check mission context
                #    continue
                print(case_k)
                
                VarLines = f'<td rowspan="2" >{case_k if k < len(cols) else "All"}</td>'
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                for model in models_list:
                    max_g = {metric:
                        max([round(files['global'][key].loc[model,metric],4) for key in files['global'].keys()]) for metric in metrics
                        }
                    max_p = {metric:
                        max([round(files['personalized'][key].loc[model,metric],4) for key in files['personalized'].keys()]) for metric in metrics
                        }
                    data = {
                        'global':'<td>Global</td>',
                        'personalized':f'<tr><td>Personalized</td>'
                    }
                    for key in files['global'].keys():
                        data['global']+= ''.join([ 
                            f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(round(files["global"][key].loc[model,metric],4) == max_g[metric])}>{round(files["global"][key].loc[model,metric],4)}</td>' 
                            for metric in metrics
                            ])
                        data['personalized']+= ''.join([ 
                            f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(round(files["personalized"][key].loc[model,metric],4) == max_p[metric])}>{round(files["personalized"][key].loc[model,metric],4)}</td>' 
                            for metric in metrics
                            ])
                    data['global']+= '</tr>'
                    data['personalized']+= '</tr>'
                    body_l[model] += (LayerLines + VarLines + data['global'] + data['personalized'])  if len(body_l[model]) == 0 else ( f'<tr >' + VarLines + data['global'] + data['personalized'])
            
            for model in models_name.keys():
                body[model]+= body_l[model]
                caption = f'<caption><h2>Legend</h2>{caption_content_lambda(model)}</caption>'
                table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{head[model]}{body_l[model]}</table>'
                htm = f'<html><head>{style}<title> Summary </title></head><body style="background-color: white;">{table_html}</body></html>'

                create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/mlna_{k}/report')
                timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
                filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/mlna_{k}/report/Summary {data_folder} for {model} '+'_'+timestr+'.html'
                _file= open(filename1,"w")
                _file.write(htm)
                _file.close()
        for model in models_name.keys():
            caption = f'<caption><h2>Legend</h2>{caption_content_lambda(model)}</caption>'
            table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{head[model]}{body[model]}</table>'
            htm = f'<html><head>{style}<title> Summary </title></head><body style="background-color: white;">{table_html}</body></html>'
            
            create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/report')
            timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
            filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/report/Summary {data_folder} for {model} '+'_'+timestr+'.html'
            _file= open(filename1,"w")
            _file.write(htm)
            _file.close()



In [12]:
get_qualitative_from_cols = lambda x:(list(set([
    var.split("__")[1] for var in [
        coll 
        for coll in [
            col 
            for col in x 
                if not (
                    ('precision' in col ) 
                    or ('accuracy' in col ) 
                    or ('recall' in col) 
                    or ('f1-score' in col)
                )
            ] 
            if ("__" in coll)
        ]
    ]
)))

In [13]:
def analyzer_launcher(outputs_name=None):
    
    result_folders = [dirnames for _, dirnames, _ in os.walk(f'{os.getcwd()}/{outputs_name}')][0]
    for dataset_name in result_folders:
        print(dataset_name)
        classic_f = [
                        load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        for file in get_filenames(
                            root_dir=f'{os.getcwd()}/{outputs_name}/{dataset_name}/data_selection_storage', 
                            func=MLN_C, 
                            verbose=False
                            )
                        ][-1]
        quali_col = get_qualitative_from_cols(classic_f.columns.to_list())
        models = model_desc()
        
        metrics_analyzer(
            cols=quali_col, 
            outputs_path=f'{os.getcwd()}/{outputs_name}/{dataset_name}', 
            cwd=os.getcwd(), 
            data_folder=dataset_name, 
            classic_metrics=classic_f, 
            models_name=models
            )
        

In [14]:
analyzer_launcher(outputs_name="outputs_19_02_2024")

AER
owner
['xgb', 'dtc', 'lrc', 'rfc']
selfemp
['xgb', 'dtc', 'lrc', 'rfc']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_19_02_2024_made_on_2024_02_22_13H/AER/plots/mlna_1/report' created successfully.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_19_02_2024_made_on_2024_02_22_13H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_19_02_2024_made_on_2024_02_22_13H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_19_02_2024_made_on_2024_02_22_13H/AER/plots/mlna_1/report' already exists.
owner±selfemp
['xgb', 'dtc', 'lrc', 'rfc']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_19_02_2024_made_on_2024_02_22_13H/AER/plots/mlna_2/report' create

In [15]:
analyzer_launcher(outputs_name="outputs_1")

AER
owner
['xgb', 'dtc', 'lrc', 'rfc']
selfemp
['xgb', 'dtc', 'lrc', 'rfc']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_1_made_on_2024_02_22_13H/AER/plots/mlna_1/report' created successfully.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_1_made_on_2024_02_22_13H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_1_made_on_2024_02_22_13H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_1_made_on_2024_02_22_13H/AER/plots/mlna_1/report' already exists.
owner±selfemp
['xgb', 'dtc', 'lrc', 'rfc']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_1_made_on_2024_02_22_13H/AER/plots/mlna_2/report' created successfully.
Directory '/Users/djiemboutie

<!DOCTYPE html>
<html>
<head>
<title>xxx</title>
<style>
.metrics > td {
writing-mode: vertical-rl;
text-orientation: sideways;
}
</style>
<table>
<tr>
<td rowspan="2" colspan="3">XGB<td>
<td colspan="4">Classic</td>
<td colspan="4">Classic - Att</td>
<td colspan="4">Classic + MLN</td>
<td colspan="4">Classic + MLN - Att</td>
</tr>
<tr class="metrics">
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
</tr>
<tr>
<td rowspan="2">MLN 1</td>
<td rowspan="2">Var 1</td>
<td>Global</td>
<td>0</td><td>1</td><td>2</td><td>3</td>
<td>4</td><td>5</td><td>6</td><td>7</td>
<td>8</td><td>9</td><td>10</td><td>11</td>
<td>12</td><td>13</td><td>14</td><td>15</td>
</tr>
<tr>
<td>Personalized</td>
<td>0</td><td>1</td><td>2</td><td>3</td>
<td>4</td><td>5</td><td>6</td><td>7</td>
<td>8</td><td>9</td><td>10</td><td>11</td>
<td>12</td><td>13</td><td>14</td><td>15</td>
</tr>
</table>

In [16]:
data_folder = 'AFB'
models_name = model_desc()
caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            'MLN': 'MultiLayer Network',
            'MLN k Layer(s)': 'MLN with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'Desc': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + Desc': f'Learning from classic dataset of {data_folder} where Desc had been added',
            'Classic + Desc - Att': f'Learning from classic dataset of {data_folder} where Desc had been added and Att removed',
            f'{x}': models_name[x]
            }.items()])
caption = f'<caption><h2>Legend</h2>{caption_content_lambda("xgb")}</caption>'
style = '<style>table, th, td {border: 1px solid black;border-collapse: collapse;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black; padding: 2px;} caption > h2 {text-align: center;} td {text-align: center;}</style>'
fake = """
<tr>
<td rowspan="2" colspan="3">XGB<td>
<td colspan="4">Classic</td>
<td colspan="4">Classic - Att</td>
<td colspan="4">Classic + MLN</td>
<td colspan="4">Classic + MLN - Att</td>
</tr>
<tr class="metrics">
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>
</tr>
<tr>
<td rowspan="2">MLN 1</td>
<td rowspan="2">Var 1</td>
<td>Global</td>
<td>0</td><td>1</td><td>2</td><td>3</td>
<td>4</td><td>5</td><td>6</td><td>7</td>
<td>8</td><td>9</td><td>10</td><td>11</td>
<td>12</td><td>13</td><td>14</td><td>15</td>
</tr>
<tr>
<td>Personalized</td>
<td>0</td><td>1</td><td>2</td><td>3</td>
<td>4</td><td>5</td><td>6</td><td>7</td>
<td>8</td><td>9</td><td>10</td><td>11</td>
<td>12</td><td>13</td><td>14</td><td>15</td>
</tr>
"""
table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption} {fake}</table>'
htm = f'<html><head>{style}<title> Summary </title></head><body style="background-color: white;">{table_html}</body></html>'

HTML(htm)

In [17]:
htm

'<html><head><style>table, th, td {border: 1px solid black;border-collapse: collapse;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black; padding: 2px;} caption > h2 {text-align: center;} td {text-align: center;}</style><title> Summary </title></head><body style="background-color: white;"><table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;"><caption><h2>Legend</h2><span><strong>MLN</strong>: MultiLayer Network</span><br><span><strong>MLN k Layer(s)</strong>: MLN with k layer(s)</span><br><span><strong>Att</strong>: Attributs or modalities of variable(s) used to build MLN</span><br><span><strong>Desc</strong>: Descriptors extracted from MLN</span><br><span><strong>Classic</strong>: Learning from classic dataset of AFB</span><br><span><strong>Classic - Att</strong>: Learning from classic dataset of AFB where Att had been removed</span><br><span><strong>Classic + Desc</strong>: Learning from classic data


<html><head><style>table, th, td {border: 1px solid black;border-collapse: collapse;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;} td {text-align: center;}</style><title> Summary </title></head><body style="background-color: white;"><table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;"><caption><h2>Legend</h2><span><strong>MLN</strong>: MultiLayer Network</span><br><span><strong>MLN k Layer(s)</strong>: MLN with k layer(s)</span><br><span><strong>Att</strong>: Attributs or modalities of variable(s) used to build MLN</span><br><span><strong>Desc</strong>: Descriptors extracted from MLN</span><br><span><strong>Classic</strong>: Learning from classic dataset of AFB</span><br><span><strong>Classic - Att</strong>: Learning from classic dataset of AFB where Att had been removed</span><br><span><strong>Classic + Desc</strong>: Learning from classic dataset of AFB where Desc had been added</span><br><span><strong>Classic + Desc - Att</strong>: Learning from classic dataset of AFB where Desc had been added and Att removed</span><br><span><strong>xgb</strong>: XGBOOST</span><br></caption> <tr class=\'dd\'><td rowspan="2" colspan="3">XGB<td><td colspan="4">Classic</td><td colspan="4">Classic - Att</td><td colspan="4">Classic + MLN</td><td colspan="4">Classic + MLN - Att</td></tr><tr class="metrics"><td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td><td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td><td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td><td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td></tr><tr><td rowspan="2">MLN 1</td><td rowspan="2">Var 1</td><td>Global</td><td>0</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td><td>9</td><td>10</td><td>11</td><td>12</td><td>13</td><td>14</td><td>15</td></tr><tr><td>Personalized</td><td>0</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td><td>9</td><td>10</td><td>11</td><td>12</td><td>13</td><td>14</td><td>15</td></tr></table></body></html>